## 📖 Teoria: Allocazione Dinamica

### Stack vs Heap

```c
// STACK: size fissa, scope limitato
uint32_t ips[100];  // Max 100 IP
                    // Distrutto a fine funzione

// HEAP: size dinamica, lifetime controllato
uint32_t *ips = malloc(n * sizeof(uint32_t));
                    // n può essere qualsiasi valore
                    // Vive finché non chiami free()
```

### Funzioni Memoria

```c
// Alloca
void *malloc(size_t size);
  → Alloca size byte non inizializzati
  → Ritorna puntatore o NULL se fallisce

// Ridimensiona
void *realloc(void *ptr, size_t new_size);
  → Cambia size di allocazione esistente
  → Copia dati se necessario
  → Ritorna nuovo puntatore o NULL

// Dealloca
void free(void *ptr);
  → Libera memoria allocata
  → ptr diventa "dangling pointer"
```

## 📖 Teoria: Growing Array

### Pattern: Raddoppia Capacità

```
Stato iniziale:
  capacity = 4
  size = 0
  ips = [_, _, _, _]

Aggiungi 4 elementi:
  size = 4
  ips = [1, 2, 3, 4]  ← PIENO!

Aggiungi 5° elemento:
  1. capacity_piena? → realloc(capacity * 2)
  2. capacity = 8
  3. ips = [1, 2, 3, 4, _, _, _, _]
  4. ips[4] = 5
  5. size = 5
```

### Perché Raddoppiare?

**Opzione 1: +1 ogni volta (male)**
```
Inserimenti: 1000
realloc chiamate: 1000  ← O(n²) copy totali!
```

**Opzione 2: *2 quando pieno (bene)**
```
Inserimenti: 1000
realloc chiamate: log₂(1000) ≈ 10  ← O(n) amortized!
Capacità: 1 → 2 → 4 → 8 → 16 → 32 → 64 → 128 → 256 → 512 → 1024
```

## 📝 Implementazione: IPSet Struct

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/05
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/05

# Copia file precedenti
cp ../04/ip_types.h .
cp ../04/ip_utils.* .

# Crea header IPSet
cat > ipset.h << 'EOF'
#ifndef IPSET_H
#define IPSET_H

#include <stdint.h>
#include <stdbool.h>
#include <stddef.h>

/**
 * IPSet - Set dinamico di indirizzi IP
 * 
 * Struttura dati per memorizzare IP generati:
 * - Array dinamico ridimensionabile
 * - Deduplicazione automatica
 * - Growing capacity (raddoppia quando pieno)
 */
typedef struct {
    uint32_t *ips;        // Array dinamico di IP
    size_t size;          // Numero elementi corrente
    size_t capacity;      // Capacità allocata
} IPSet;

// Inizializzazione e cleanup
IPSet* ipset_create(size_t initial_capacity);
void ipset_free(IPSet *set);

// Operazioni
bool ipset_add(IPSet *set, uint32_t ip);
bool ipset_contains(const IPSet *set, uint32_t ip);
void ipset_clear(IPSet *set);

// Getters
size_t ipset_size(const IPSet *set);
size_t ipset_capacity(const IPSet *set);
uint32_t ipset_get(const IPSet *set, size_t index);

#endif // IPSET_H
EOF

echo "✅ ipset.h creato"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/05

cat > ipset.c << 'EOF'
#include "ipset.h"
#include <stdlib.h>
#include <string.h>

/**
 * Crea nuovo IPSet
 * 
 * @param initial_capacity Capacità iniziale array
 * @return Puntatore a IPSet allocato, o NULL se errore
 */
IPSet* ipset_create(size_t initial_capacity) {
    // Capacità minima: 8
    if (initial_capacity < 8) {
        initial_capacity = 8;
    }
    
    // Alloca struct
    IPSet *set = malloc(sizeof(IPSet));
    if (!set) return NULL;
    
    // Alloca array IP
    set->ips = malloc(initial_capacity * sizeof(uint32_t));
    if (!set->ips) {
        free(set);  // Cleanup parziale!
        return NULL;
    }
    
    set->size = 0;
    set->capacity = initial_capacity;
    
    return set;
}

/**
 * Libera memoria IPSet
 * 
 * @param set IPSet da distruggere (può essere NULL)
 */
void ipset_free(IPSet *set) {
    if (!set) return;  // NULL-safe
    
    free(set->ips);    // Libera array
    free(set);         // Libera struct
}

/**
 * Verifica se IP esiste già nel set
 * 
 * Algoritmo: ricerca lineare O(n)
 * (Per set grandi, usare hash table o binary search)
 */
bool ipset_contains(const IPSet *set, uint32_t ip) {
    if (!set) return false;
    
    for (size_t i = 0; i < set->size; i++) {
        if (set->ips[i] == ip) {
            return true;
        }
    }
    return false;
}

/**
 * Espande capacità array (raddoppia)
 * 
 * @return true se successo, false se errore
 */
static bool ipset_grow(IPSet *set) {
    size_t new_capacity = set->capacity * 2;
    
    // realloc può spostare memoria
    uint32_t *new_ips = realloc(set->ips, 
                                 new_capacity * sizeof(uint32_t));
    if (!new_ips) {
        return false;  // Errore allocazione
    }
    
    set->ips = new_ips;
    set->capacity = new_capacity;
    return true;
}

/**
 * Aggiunge IP al set (se non duplicato)
 * 
 * Algoritmo:
 * 1. Verifica duplicato (ipset_contains)
 * 2. Se capacity piena → grow (realloc)
 * 3. Aggiungi IP
 * 
 * @return true se aggiunto, false se duplicato o errore
 */
bool ipset_add(IPSet *set, uint32_t ip) {
    if (!set) return false;
    
    // Dedup: se esiste già, skip
    if (ipset_contains(set, ip)) {
        return false;
    }
    
    // Se pieno, espandi
    if (set->size >= set->capacity) {
        if (!ipset_grow(set)) {
            return false;  // Errore realloc
        }
    }
    
    // Aggiungi IP
    set->ips[set->size++] = ip;
    return true;
}

/**
 * Svuota set (mantiene capacità)
 */
void ipset_clear(IPSet *set) {
    if (set) {
        set->size = 0;
    }
}

// Getters

size_t ipset_size(const IPSet *set) {
    return set ? set->size : 0;
}

size_t ipset_capacity(const IPSet *set) {
    return set ? set->capacity : 0;
}

uint32_t ipset_get(const IPSet *set, size_t index) {
    if (!set || index >= set->size) {
        return 0;  // Out of bounds
    }
    return set->ips[index];
}
EOF

echo "✅ ipset.c implementato"

## ✅ Test IPSet

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/05

cat > test_ipset.c << 'EOF'
#include <stdio.h>
#include "ipset.h"
#include "ip_utils.h"

void test_create_free() {
    printf("=== TEST ipset_create() ===\n");
    
    IPSet *set = ipset_create(4);
    
    if (set) {
        printf("  ✓ Set creato\n");
        printf("  Size: %zu\n", ipset_size(set));
        printf("  Capacity: %zu\n\n", ipset_capacity(set));
        ipset_free(set);
    } else {
        printf("  ✗ Errore allocazione\n\n");
    }
}

void test_add_basic() {
    printf("=== TEST ipset_add() Basic ===\n");
    
    IPSet *set = ipset_create(4);
    
    uint32_t ip1, ip2;
    string_to_ip("192.168.1.1", &ip1);
    string_to_ip("8.8.8.8", &ip2);
    
    bool added1 = ipset_add(set, ip1);
    bool added2 = ipset_add(set, ip2);
    
    printf("  192.168.1.1 aggiunto: %s\n", added1 ? "✓" : "✗");
    printf("  8.8.8.8 aggiunto: %s\n", added2 ? "✓" : "✗");
    printf("  Size: %zu\n\n", ipset_size(set));
    
    ipset_free(set);
}

void test_deduplication() {
    printf("=== TEST Deduplicazione ===\n");
    
    IPSet *set = ipset_create(4);
    
    uint32_t ip;
    string_to_ip("10.0.0.1", &ip);
    
    bool add1 = ipset_add(set, ip);
    bool add2 = ipset_add(set, ip);  // Duplicato!
    bool add3 = ipset_add(set, ip);  // Duplicato!
    
    printf("  Prima aggiunta: %s\n", add1 ? "✓ OK" : "✗ Rifiutato");
    printf("  Seconda (dup): %s\n", add2 ? "✓ OK" : "✗ Rifiutato");
    printf("  Terza (dup): %s\n", add3 ? "✓ OK" : "✗ Rifiutato");
    printf("  Size finale: %zu (atteso: 1)\n\n", ipset_size(set));
    
    ipset_free(set);
}

void test_contains() {
    printf("=== TEST ipset_contains() ===\n");
    
    IPSet *set = ipset_create(4);
    
    uint32_t ip1, ip2;
    string_to_ip("192.168.1.1", &ip1);
    string_to_ip("8.8.8.8", &ip2);
    
    ipset_add(set, ip1);
    
    printf("  192.168.1.1 presente: %s\n", 
           ipset_contains(set, ip1) ? "✓" : "✗");
    printf("  8.8.8.8 presente: %s\n\n", 
           ipset_contains(set, ip2) ? "✓" : "✗");
    
    ipset_free(set);
}

void test_growing() {
    printf("=== TEST Growing Array ===\n");
    
    IPSet *set = ipset_create(2);  // Capacity minima
    
    printf("  Capacity iniziale: %zu\n\n", ipset_capacity(set));
    
    // Aggiungi 10 IP univoci
    for (uint32_t i = 0; i < 10; i++) {
        uint32_t ip = 0x08080800 + i;  // 8.8.8.x
        ipset_add(set, ip);
        
        printf("  Dopo %2u add: size=%zu, capacity=%zu", 
               i + 1, ipset_size(set), ipset_capacity(set));
        
        if (i > 0 && ipset_capacity(set) != ipset_capacity(set)) {
            printf(" ← GROW!");
        }
        printf("\n");
    }
    
    printf("\n  Progressione capacity: 8 → 16\n");
    printf("  (capacity minima forza 8, non 2)\n\n");
    
    ipset_free(set);
}

void test_get_iterate() {
    printf("=== TEST ipset_get() ===\n");
    
    IPSet *set = ipset_create(8);
    
    const char *ips_str[] = {
        "8.8.8.8",
        "1.1.1.1",
        "192.168.1.1"
    };
    
    // Aggiungi
    for (int i = 0; i < 3; i++) {
        uint32_t ip;
        string_to_ip(ips_str[i], &ip);
        ipset_add(set, ip);
    }
    
    // Itera
    printf("  Contenuto set:\n");
    for (size_t i = 0; i < ipset_size(set); i++) {
        uint32_t ip = ipset_get(set, i);
        char buf[16];
        ip_to_string(ip, buf);
        printf("    [%zu] %s\n", i, buf);
    }
    printf("\n");
    
    ipset_free(set);
}

void test_clear() {
    printf("=== TEST ipset_clear() ===\n");
    
    IPSet *set = ipset_create(8);
    
    // Aggiungi 5 IP
    for (uint32_t i = 0; i < 5; i++) {
        ipset_add(set, 0x08080800 + i);
    }
    
    printf("  Prima: size=%zu, capacity=%zu\n", 
           ipset_size(set), ipset_capacity(set));
    
    ipset_clear(set);
    
    printf("  Dopo clear: size=%zu, capacity=%zu\n", 
           ipset_size(set), ipset_capacity(set));
    printf("  (capacity mantenuta per riuso)\n\n");
    
    ipset_free(set);
}

void test_memory_lifecycle() {
    printf("=== TEST Memory Lifecycle ===\n\n");
    
    printf("  1. IPSet *set = ipset_create(8);\n");
    printf("     → malloc(sizeof(IPSet))\n");
    printf("     → malloc(8 * sizeof(uint32_t))\n");
    printf("     → 2 allocazioni heap\n\n");
    
    printf("  2. Aggiungi 9° elemento:\n");
    printf("     → size >= capacity\n");
    printf("     → realloc(ips, 16 * sizeof(uint32_t))\n");
    printf("     → Possibile spostamento memoria\n\n");
    
    printf("  3. ipset_free(set);\n");
    printf("     → free(set->ips)\n");
    printf("     → free(set)\n");
    printf("     → Tutta memoria liberata\n\n");
    
    printf("  ⚠️ Regola d'oro:\n");
    printf("     Ogni malloc/realloc → un free!\n");
}

int main() {
    test_create_free();
    test_add_basic();
    test_deduplication();
    test_contains();
    test_growing();
    test_get_iterate();
    test_clear();
    test_memory_lifecycle();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_ipset ip_utils.c ipset.c test_ipset.c && ./test_ipset

## 🔍 Analisi: Gestione Errori

### Cleanup Parziale

```c
IPSet* ipset_create(size_t cap) {
    IPSet *set = malloc(sizeof(IPSet));
    if (!set) return NULL;  // ← Errore 1
    
    set->ips = malloc(cap * sizeof(uint32_t));
    if (!set->ips) {
        free(set);  // ← CRITICO: pulisci set!
        return NULL;
    }
    
    return set;
}
```

**Senza cleanup:**
```
malloc(set) → OK
malloc(ips) → FAIL
return NULL
             ↓
         MEMORY LEAK! (set non liberato)
```

### NULL-Safe Free

```c
void ipset_free(IPSet *set) {
    if (!set) return;  // ← Sicuro chiamare con NULL
    
    free(set->ips);    // free(NULL) è safe!
    free(set);
}
```

## 🔍 Analisi: realloc Gotchas

### ❌ ERRORE Comune

```c
// SBAGLIATO!
set->ips = realloc(set->ips, new_size);
if (!set->ips) {
    // PROBLEMA: se realloc fallisce, ritorna NULL
    // ma set->ips originale è perso!
    // → MEMORY LEAK
}
```

### ✅ CORRETTO

```c
// Usa variabile temporanea
uint32_t *new_ips = realloc(set->ips, new_size);
if (!new_ips) {
    // set->ips ancora valido!
    // Puoi continuare con capacity vecchia
    return false;
}
set->ips = new_ips;  // Aggiorna solo se successo
```

## 📊 Complessità

| Operazione | Complessità | Note |
|------------|-------------|------|
| `ipset_create` | O(1) | Allocazione costante |
| `ipset_free` | O(1) | Deallocazione costante |
| `ipset_add` | O(n) | Dedup richiede scan |
| `ipset_contains` | O(n) | Ricerca lineare |
| `ipset_get` | O(1) | Accesso array diretto |
| `ipset_clear` | O(1) | Solo reset size |
| **Grow** | O(1) amortized | Raddoppio logaritmico |

### Miglioramenti Possibili

```
1. Hash table → O(1) add/contains
2. Sorted array → O(log n) contains (binary search)
3. Bloom filter → O(1) probabilistic contains
```

## 📚 Recap

### ✅ Implementazione completa:

```c
IPSet {
    uint32_t *ips;   // Array dinamico
    size_t size;     // # elementi
    size_t capacity; // # allocati
}
```

**Funzioni:**
1. `ipset_create()` - malloc struct + array
2. `ipset_free()` - free array + struct
3. `ipset_add()` - dedup + grow + insert
4. `ipset_contains()` - ricerca O(n)
5. `ipset_clear()` - reset size
6. Getters: size, capacity, get

### 🎓 Concetti chiave:

- **Growing array**: raddoppia quando pieno
- **realloc pattern**: usa temp var
- **Cleanup parziale**: free on error
- **NULL-safe**: controlla puntatori
- **Deduplicazione**: ipset_contains prima di add

### 🔑 Memory safety:

```
Ogni malloc → un free
Ogni realloc → temp var
Errore → cleanup parziale
free(NULL) → sempre safe
```

### ➡️ Prossimo Notebook

**Notebook 6: Parser CIDR/Range**
- `parse_cidr()` - 192.168.1.0/24
- `parse_range()` - 10.0.0.1-10.0.0.50
- `parse_single()` - 8.8.8.8
- `expand_range()` - genera tutti IP nel range
- `strtok` e string parsing

---

## 🎉 Notebook 5 Completato!

**Funzioni**: 8 operazioni IPSet  
**Concetti**: malloc/realloc/free, growing array, memory safety

Continua con il Notebook 6! 🚀